### Configure Notebook

#### Import Libraries

In [1]:
import base64
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from io import BytesIO
import pandas as pd
import numpy as np

#### Define Functions

In [2]:
def load_encrypted_dataset(encrypted_file, decryption_key_path):
    ''' A function uses a decryption key to decrypt a file.

    Parameters:
    encrypted_file (str): The path to a file which has been encrypted.
    decryption_key_path (str): The path to the corresponding decryption key.

    Returns:
    decrypted_dataset (Pandas DataFrame): A decrypted version of the encrypted file.
    '''
    
    file = open(decryption_key_path, 'rb') # Open file.
    decryption_key = file.read() # Read decryption key.
    file.close() # Close file.
       
    fernet = Fernet(decryption_key)
    
    with open(encrypted_file, 'rb') as f:
        encrypted_file = f.read()
    
    decrypted_file = fernet.decrypt(encrypted_file) # Decrypt file.
    decrypted_dataset = pd.read_csv(BytesIO(decrypted_file), index_col=0) # Convert bytes data to Pandas DataFrame.
    
    return decrypted_dataset

### Test

#### Generate Random Dataset

In [3]:
unsecure_df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

unsecure_df.to_csv('unsecure.csv') # Export corrected dataset.

unsecure_df

,A,B,C,D
0,63,68,35,90
1,80,28,66,73
2,83,70,28,41
3,37,88,23,88
4,43,13,89,66
...,...,...,...,...
95,95,70,25,8
96,65,41,64,96
97,14,47,40,24
98,98,81,65,66


#### Generate Encryption Key

In [4]:
password_provided = 'password' # Provide password.
password = password_provided.encode() # Encode password.

salt = b"randomsalt" # Provide salt key.

kdf = PBKDF2HMAC(algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000,
                backend=default_backend())

key = base64.urlsafe_b64encode(kdf.derive(password))

file = open(f'encryption_key.key', 'wb')
file.write(key)
file.close()

#### Encrypt Dataset

In [5]:
file = open(f'encryption_key.key', 'rb')
key = file.read()
file.close()

# Open the file to encrypt
with open(f'unsecure.csv', 'rb') as f: # open dataset file.
    dataset = f.read()

fernet = Fernet(key)
encrypted_dataset = fernet.encrypt(dataset)

# Write the encrypted file
with open(f'secure.csv.encrypted', 'wb') as f: # Encrypt dataset csv file.
    f.write(encrypted_dataset)
    
encrypted_dataset

b'gAAAAABh8DEW67ijVxmQ0N8Dl6ZStpg8oqiTQ1pORRGUKKBrE8_K9dpA-a4hAR1qybGRgLTcvbXdpPdQeP-Ize5Y66nio3fnFAuQPd6n1V-ZK4t4crzQSxgWTN7CMj7iND3DDGHC7aZzhczVtnKzHGrBV1UGOtG_JAXodVo2OXD1tf1ot2yFSk7aZpLeS4k9YG5-PRSEAPypo_Z4Xb98ah0NgWYJLhsBWukBX5iQKw1KoxKRxPXZnjSvtVkbkM8WqMIpRWgT_m2VX116qKFhkpKUXI8qEBVqWNzB1VioMcNUdY3_yNanzMGApv6Oi7z5IMOxuzAcGRaAtNe_UMVGk6vMW0ihn3o4BMxh0rcsktcwJ1GnDUGJPbmgk2HPboXfr2WvszqY-uOz2oUISQdO7TJAPIOc-WhNyBXkz7en0XFq0YWFh6r7AAFyRiAoEAx5yMcpqNSoW7lJxmp6M7NR3CIjojovBuDUKWmQbCt5oxnUrgWwNGZ-RctMmOQUY32HlcFI_8pXXr_KbdZDCP_v68lLVAcdUgp4Bpq5If03m-qj7uJ8zy0qEKIoLJ2Af-LriZev_tGf9zRrh8oqXIE5iyiFlWNk7VsAihC7Gb5Bf-1KF2vTQq5SbBvNPphpRU05Q2wpPcM9-GpM2GsY5z3v_7cOkTtTduOFX7fMFgVUnaTAcryRhLLRzZiH6krTVhLjAKvDAqai9DvoeAABDQuqwwXAl_C3VwvC0a_mJvN4xniNKzCTN700ZCsAdFBUsAc-5SSJZ0aG_dbStY--GbgIKmANQXnv7YEwWmReFkMlS1ztJgGmNTdoKLVvMFe7YWAdSchNgbNSbGYgzLjjg1Px4H7p8qJtrF_i2ooskU_x4Z37u0wEvddafpm2QINKnvrCIJdtbbGnhH-QzJG8GlmqqC9-u5Q-HqnOWf0uJBT5sSMvRkd2oWuZJQ_1UxmMf4TNXuzX4TBXkA0NQCUe1uMe2LGkI-t4X1FogyPu8a

#### Decrypt Dataset

In [6]:
decrypted_dataset = load_encrypted_dataset(encrypted_file='secure.csv.encrypted', decryption_key_path='encryption_key.key')

decrypted_dataset

,A,B,C,D
0,63,68,35,90
1,80,28,66,73
2,83,70,28,41
3,37,88,23,88
4,43,13,89,66
...,...,...,...,...
95,95,70,25,8
96,65,41,64,96
97,14,47,40,24
98,98,81,65,66
